In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf

In [ ]:
from gensim.models import FastText

In [ ]:
import spacy

In [ ]:
import string

In [ ]:
def clean_text(text):
    text = text.replace("\n"," ")
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text
    

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
def lemmatize(text):
    doc = nlp(text)
    lemmas = [token.lemma_.lower() if token.lemma_ != '-PRON-' else token.lower_ for token in doc]
    return " ".join(lemmas)
    

In [ ]:
!nvidia-smi

In [ ]:
df = pd.read_csv("/kaggle/input/commonlitreadabilityprize/train.csv")

In [ ]:
df.head()

In [ ]:
df = df.drop(["standard_error"],axis=1)

In [ ]:
df

In [ ]:
doc = nlp(df["excerpt"][0])

In [ ]:
df["excerpt"][1]

In [ ]:
lemmatize(df["excerpt"][1])

In [ ]:
import random

In [ ]:
import tensorflow as tf

In [ ]:
ind = random.randint(1,2834-5)

df.iloc[ind].target,df.iloc[ind].excerpt

In [ ]:
df.describe()

In [ ]:
import re
import string

In [ ]:
df

In [ ]:
df["excerpt"] = df["excerpt"].apply(lambda text: clean_text(text))

In [ ]:
doc2 = nlp(df["excerpt"][0]).vector

In [ ]:
doc2.shape

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
df["length"] = df["excerpt"].apply(lambda text: len(text.split()))

In [ ]:
df.describe()

In [ ]:
np.percentile(df["length"],95)

In [ ]:
max_length = 199

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df["excerpt"],df["target"],test_size=0.2,random_state=42)

In [ ]:
from keras import initializers

In [ ]:
vectorizer = TextVectorization()

In [ ]:
vectorizer.adapt(X_train.to_numpy())

In [ ]:
num_tokens = len(vectorizer.get_vocabulary())
embedding_dim = len(nlp("The").vector)

In [ ]:
#initialize the embedding matrix
embedding_matrix = np.zeros((num_tokens,embedding_dim))

In [ ]:
for index,word in enumerate(vectorizer.get_vocabulary()):
    embedding_matrix[index] = nlp(word).vector


In [ ]:
embedding_matrix[1]

In [ ]:
embedding = tf.keras.layers.Embedding(num_tokens,embedding_dim,embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),trainable=False)

In [ ]:
tf.not_equal([10,123,345,355,0,0,0,0],0)

In [ ]:
from tensorflow.keras import layers
from tensorflow import keras

In [ ]:
import tensorflow as tf

In [ ]:
from keras import backend as K

def swish(x, beta=1.0):
    return x * K.sigmoid(beta * x)
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish': Activation(swish)})

In [ ]:
inputs = layers.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(25,return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(25,return_sequences=True))(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(32,activation="swish")(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(32,activation="swish")(x)
outputs = layers.Dense(1)(x)
model_8 = tf.keras.Model(inputs,outputs)

In [ ]:
model_8.summary()

In [ ]:
model_8.compile(loss="mse",optimizer=tf.keras.optimizers.Adam(),metrics=tf.keras.metrics.RootMeanSquaredError())

In [ ]:
import datetime

In [ ]:
def create_tensorboard_callback(dir_name,experiment_name):
  """
  Creates a TensorBoard callback instand to store log files.
  Stores log files with the filepath:
    "dir_name/experiment_name/current_datetime/"
  Args:
    dir_name: target directory to store TensorBoard log files
    experiment_name: name of experiment directory (e.g. efficientnet_model_1)
  """
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

In [ ]:
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath="checkpoint/check.ckpt",monitor="val_root_mean_squared_error",save_best_only=True,save_weights_only=True)

In [ ]:
model_8.summary()

In [ ]:
model_8.fit(X_train,y_train,epochs=8,validation_data=(X_test,y_test),callbacks=[create_tensorboard_callback("CommonLit","SpaCy Embeddings-BiLSTM_NOLemmas"),model_checkpoint])

In [ ]:
df.iloc[0].excerpt

In [ ]:
df_test = pd.read_csv("/kaggle/input/commonlitreadabilityprize/test.csv")

In [ ]:
df_test

In [ ]:
preds = model_8.predict(df_test["excerpt"])

In [ ]:
preds

In [ ]:
submissions = pd.read_csv("/kaggle/input/commonlitreadabilityprize/sample_submission.csv")

In [ ]:
submissions

In [ ]:
submissions["target"] = preds

In [ ]:
submissions

In [ ]:
submissions.to_csv("/kaggle/working/submission.csv",index=False)

<a href="CommonLit2Checkpo8int.zip/"> Download File </a>